In [1]:
#Goes to dataset directory to see subfolder names and their count

import os

folder_name = 'dataset'

current_directory = os.getcwd()

folder_path = os.path.join(current_directory, folder_name)

if os.path.exists(folder_path) and os.path.isdir(folder_path):
    subfolders = [name for name in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, name))]
    
    num_subfolders = len(subfolders)
    
    print(f'Total number of subfolders in "{folder_name}": {num_subfolders}')
else:
    print(f'The folder "{folder_name}" does not exist in the current directory.')


Total number of subfolders in "dataset": 3


In [2]:
import math
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.utils.model_zoo as model_zoo



class Fire(nn.Module):

    def __init__(self, inplanes, squeeze_planes,
                 expand1x1_planes, expand3x3_planes):
        super(Fire, self).__init__()
        self.inplanes = inplanes
        self.squeeze = nn.Conv2d(inplanes, squeeze_planes, kernel_size=1)
        self.squeeze_activation = nn.ReLU(inplace=True)
        self.expand1x1 = nn.Conv2d(squeeze_planes, expand1x1_planes,
                                   kernel_size=1)
        self.expand1x1_activation = nn.ReLU(inplace=True)
        self.expand3x3 = nn.Conv2d(squeeze_planes, expand3x3_planes,
                                   kernel_size=3, padding=1)
        self.expand3x3_activation = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.squeeze_activation(self.squeeze(x))
        return torch.cat([
            self.expand1x1_activation(self.expand1x1(x)),
            self.expand3x3_activation(self.expand3x3(x))
        ], 1)


class SqueezeNet(nn.Module):

    def __init__(self,  num_classes=10):
        super(SqueezeNet, self).__init__()
        
        self.num_classes = num_classes
        
        self.features = nn.Sequential(
                nn.Conv2d(3, 64, kernel_size=3, stride=2),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
                Fire(64, 16, 64, 64),
                Fire(128, 16, 64, 64),
                nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
                Fire(128, 32, 128, 128),
                Fire(256, 32, 128, 128),
                nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
                Fire(256, 48, 192, 192),
                Fire(384, 48, 192, 192),
                Fire(384, 64, 256, 256),
                Fire(512, 64, 256, 256),
            )
        
        final_conv = nn.Conv2d(512, self.num_classes, kernel_size=1)
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            final_conv,
            nn.ReLU(inplace=True),
            nn.AvgPool2d(13)
        )

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                if m is final_conv:
                    init.normal(m.weight.data, mean=0.0, std=0.01)
                else:
                    init.kaiming_uniform(m.weight.data)
                if m.bias is not None:
                    m.bias.data.zero_()

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x.view(x.size(0), self.num_classes)




model = SqueezeNet()
model

/tmp/ipykernel_10339/182418824.py:68: UserWarning: nn.init.kaiming_uniform is now deprecated in favor of nn.init.kaiming_uniform_.
  init.kaiming_uniform(m.weight.data)
/tmp/ipykernel_10339/182418824.py:66: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  init.normal(m.weight.data, mean=0.0, std=0.01)


SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (5): MaxPool2d

In [3]:
#Loads images after transforming sizes of each image such that it fits the model
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor(),          
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  
])

dataset_root = './dataset'

dataset = ImageFolder(root=dataset_root, transform=transform)

num_classes = len(dataset.classes)

batch_size = 7

dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [4]:
#Saves the model  in .pth format
import torch
import torch.nn as nn
import torch.optim as optim
import os
os.environ["OMP_NUM_THREADS"] = "1"

criterion = nn.CrossEntropyLoss()

learning_rate = 0.001
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)               

num_epochs = 20

for epoch in range(num_epochs):
    for batch in dataloader:
        inputs, labels = batch
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

torch.save(model.state_dict(), 'squeezenet_model.pth')



In [5]:
#converts the .pth format to .onnx format
import torch
model = SqueezeNet()
model.load_state_dict(torch.load('squeezenet_model.pth'))
model.eval()  
dummy_input = torch.randn(1, 3, 224, 224)  
onnx_filename = 'squeezenet2_model.onnx'  
torch.onnx.export(model, dummy_input, onnx_filename, verbose=True)


Exported graph: graph(%input.1 : Float(1, 3, 224, 224, strides=[150528, 50176, 224, 1], requires_grad=0, device=cpu),
      %features.0.weight : Float(64, 3, 3, 3, strides=[27, 9, 3, 1], requires_grad=1, device=cpu),
      %features.0.bias : Float(64, strides=[1], requires_grad=1, device=cpu),
      %features.3.squeeze.weight : Float(16, 64, 1, 1, strides=[64, 1, 1, 1], requires_grad=1, device=cpu),
      %features.3.squeeze.bias : Float(16, strides=[1], requires_grad=1, device=cpu),
      %features.3.expand1x1.weight : Float(64, 16, 1, 1, strides=[16, 1, 1, 1], requires_grad=1, device=cpu),
      %features.3.expand1x1.bias : Float(64, strides=[1], requires_grad=1, device=cpu),
      %features.3.expand3x3.weight : Float(64, 16, 3, 3, strides=[144, 9, 3, 1], requires_grad=1, device=cpu),
      %features.3.expand3x3.bias : Float(64, strides=[1], requires_grad=1, device=cpu),
      %features.4.squeeze.weight : Float(16, 128, 1, 1, strides=[128, 1, 1, 1], requires_grad=1, device=cpu),
    

/tmp/ipykernel_10339/182418824.py:68: UserWarning: nn.init.kaiming_uniform is now deprecated in favor of nn.init.kaiming_uniform_.
  init.kaiming_uniform(m.weight.data)
/tmp/ipykernel_10339/182418824.py:66: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  init.normal(m.weight.data, mean=0.0, std=0.01)


In [6]:
#shows the canvas where it turns on webcam and takes input for testing 
#also shows probabilites of each class

import cv2
import numpy as np
import torch
import torch.nn as nn
from PIL import Image
import torchvision.transforms as transforms

model = SqueezeNet()
model.load_state_dict(torch.load('squeezenet_model.pth'))
model.eval()


import os

dataset_folder = "/home/outbreakkp/Desktop/imgclass/dataset"
class_mapping = {}

class_names = [d for d in os.listdir(dataset_folder) if os.path.isdir(os.path.join(dataset_folder, d))]

for i, class_name in enumerate(class_names):
    class_mapping[i] = class_name
# # Prompt the user to enter class names for each class
# for i in range(num_classes):
#     class_name = input(f"Enter class name for class {i + 1}: ")
#     class_mapping[i] = class_name

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    frame_tensor = transform(Image.fromarray(frame_rgb))

    with torch.no_grad():
        output = model(frame_tensor.unsqueeze(0))
        probabilities = torch.softmax(output, dim=1).squeeze(0)

    cv2.putText(frame, "Probabilities:", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    for i, class_name in class_mapping.items():
        prob = probabilities[i]
        prob_text = f"{class_name}: {prob.item():.4f}"
        cv2.putText(frame, prob_text, (10, 60 + i * 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    cv2.imshow('Webcam', frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

/tmp/ipykernel_10339/182418824.py:68: UserWarning: nn.init.kaiming_uniform is now deprecated in favor of nn.init.kaiming_uniform_.
  init.kaiming_uniform(m.weight.data)
/tmp/ipykernel_10339/182418824.py:66: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  init.normal(m.weight.data, mean=0.0, std=0.01)
[ WARN:0@0.017] global /croot/opencv-suite_1691620365762/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in ""
libGL error: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: iris
libGL error: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search path

In [7]:
import os

dataset_folder = "/home/outbreakkp/Desktop/imgclass/dataset"
class_mapping = {}

class_names = [d for d in os.listdir(dataset_folder) if os.path.isdir(os.path.join(dataset_folder, d))]

for i, class_name in enumerate(class_names):
    class_mapping[i] = class_name


In [8]:
from PIL import Image
import torch
import torchvision.transforms as transforms

image_path = 'photo_7_2023-11-02_12-01-19.jpg'
image = Image.open(image_path)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
image = transform(image)

with torch.no_grad():
    output = model(image.unsqueeze(0))
    probabilities = torch.softmax(output, dim=1)
    
class_probabilities = probabilities[0].tolist()

# Create a dictionary to associate class names with probabilities
class_probabilities_dict = {class_name: probability for class_name, probability in zip(class_names, class_probabilities)}

# Print the class names and their corresponding probabilities
for class_name, probability in class_probabilities_dict.items():
    print(f"{class_name}: {probability:.2f}")

class1: 0.23
class2: 0.25
class3: 0.34


In [9]:
len(class_names)

3

In [10]:
import json

In [11]:

json_output = json.dumps(class_probabilities_dict, indent=4)

print(json_output)



{
    "class1": 0.22512489557266235,
    "class2": 0.24998433887958527,
    "class3": 0.33869683742523193
}


In [12]:
# class_mapping = {
#     0: "Class1",
#     1: "Class2",
    
# }

# predicted_class_name = class_mapping[predicted_class]

In [13]:
# for class_idx in range(len(class_mapping)):
#     class_name = class_mapping[class_idx]
#     probability = probabilities[0][class_idx].item()
#     print(f"Probability of {class_name}: {probability:.4f}")

In [14]:
# onnx_filename

In [15]:
# import onnx
# import torch

# from onnxruntime import InferenceSession
# json_wla='model.json'
# onnx_model=onnx.load(onnx_filename)
# onnx_session=InferenceSession(onnx_filename)
# with open(json_wla, 'w') as f:
#     f.write(onnx_model.SerializeToString())

# print("Model saved as ",json_wla)

